In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import math
import datetime
from psaw import PushshiftAPI
from __future__ import annotations
import os
import json

In [4]:

def get_names(path: str = '' ) -> dict[str, list[str]]:
    """
    Reads the character data base characters.json.
    """
    with open(os.path.join(path, "characters.json"), "r") as f:
        characters = json.load(f)
    return characters

def find_names(doc: str, name_db: dict[str, list[str]]) -> list[str]:
    """
    Returns a list of all characters that were present in the document.
    Characters are returned as full names and not as they were found.
    """
    out = list()
    for name, keys in name_db.items():
        for k in keys:
            if k in doc:
                out.append(name)
                break
    return out


In [5]:
api = PushshiftAPI()
features = ["title", "id", "score", "author", "created_utc", "selftext"]
subreddit = "asoiaf"
date1 = int(datetime.datetime(2015,7,16).timestamp())
date2 = int(datetime.datetime(2019,4,14).timestamp())
gen = api.search_submissions(subreddit = subreddit, after = date1, before = date2, q='', filter = features, limit = 3 * 10**5)

In [ ]:
results = list(gen)

C:\Users\Oskar Wiese\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 502
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Oskar Wiese\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [ ]:

ids, title, score, author, created_utc, selftext = [], [], [], [], [], []
for i in range(len(results)):
    try:
        selftext.append(results[i].d_['selftext'])
        ids.append(i)
        title.append(results[i].d_['title'])
        score.append(results[i].d_['score'])
        created_utc.append(results[i].d_['created_utc'])
        author.append(results[i].d_['author'])
    except KeyError:
        pass
        

data = pd.DataFrame(
{
    "id" : ids,
    "title" : title,
    "score" : score,
    "created" : created_utc,
    "author" : author,
    "selftext" : selftext
}, index = None)

In [ ]:
# To see amount of posts that acutally has text in selftext
def get_usefull_data():
    text_list = [text for text in data['selftext'] if text != '' ]
    text_list = [text for text in text_list if text != '[removed]' ]
    return len(text_list)

def get_postw_names():
    text_list = [text for text in data['names'] if text != [] ]
    return len(text_list)
get_usefull_data()

In [ ]:
def mentioned_chars(df: pd.DataFrame) -> pd.DataFrame:
    names = get_names()
    row_name_finder = lambda row: find_names(row["selftext"], names)
    
    df["names"] = df.apply(row_name_finder, axis=1)
    return df
test = mentioned_chars(data)
test

In [ ]:
get_postw_names()

In [ ]:
data.to_csv('asoiaf_data.csv',index=True)